In [174]:
import dask_image.ndfilters
import flox.xarray
import numpy as np
import xarray as xr
import echopype as ep
from echopype.clean.utils import (
    extract_dB,
    pool_Sv,
    index_binning_pool_Sv,
    index_binning_downsample_upsample_along_depth,
    downsample_upsample_along_depth,
)
from echopype.utils.compute import _lin2log, _log2lin

In [175]:
# Open raw, calibrate, and add depth
ed = ep.open_raw("echopype/test_data/ek60/from_echopy/JR161-D20061118-T010645.raw", sonar_model="EK60")
ds_Sv = ep.calibrate.compute_Sv(ed)
ds_Sv = ep.consolidate.add_depth(ds_Sv)

# Select Sv subset
ds_Sv = ds_Sv.isel(ping_time=slice(100,120), range_sample=slice(990,1020))

# Set window args
depth_bin = 1
num_side_pings = 2
exclude_above = 186
range_var = "depth"
chunk_dict = {}

# Compute pooled Sv using index binning
pooled_Sv = index_binning_pool_Sv(
    ds_Sv, np.nanmean, depth_bin, num_side_pings, exclude_above, range_var, chunk_dict
).compute()

# Compute `ds_Sv` prior to using it for manual testing
ds_Sv = ds_Sv.compute()

# Iterate through channels
for channel_index in range(len(ds_Sv["channel"])):
    # Grab single channel Sv
    chan_ds_Sv = ds_Sv.isel(channel=channel_index)

    # Compute number of range sample indices that are needed to encapsulate the `depth_bin`
    # value per channel.
    num_range_sample_indices = np.ceil(
        depth_bin / np.nanmean(np.diff(chan_ds_Sv["depth"], axis=1), axis=(0,1))
    ).astype(int)

    # Compute min and max values
    range_sample_min = ds_Sv["range_sample"].min()
    range_sample_max = ds_Sv["range_sample"].max()
    ping_time_index_min = 0
    ping_time_index_max = len(ds_Sv["ping_time"])

    # Create ping time indices array
    ping_time_indices = xr.DataArray(
        np.arange(len(chan_ds_Sv["ping_time"]), dtype=int),
        dims=["ping_time"],
        coords=[ds_Sv["ping_time"]],
        name="ping_time_indices",
    )

    # Check correct binning and aggregation values
    for ping_time_index in range(len(chan_ds_Sv["ping_time"])):
        for range_sample_index in range(len(chan_ds_Sv["range_sample"])):
            # Grab pooled value
            pooled_value = pooled_Sv.isel(
                channel=channel_index,
                ping_time=ping_time_index,
                range_sample=range_sample_index
            ).data
            if not np.isnan(pooled_value):
                # Check that manually computed pool value matches Dask-Image's
                # generic filter output
                assert np.isclose(
                    pooled_value,
                    _lin2log(
                        _log2lin(
                            ds_Sv["Sv"].isel(
                                channel=channel_index,
                                ping_time=slice(
                                    ping_time_index - num_side_pings,
                                    ping_time_index + 1 + num_side_pings
                                ),
                                range_sample=slice(
                                    range_sample_index - num_range_sample_indices,
                                    range_sample_index + 1 + num_range_sample_indices
                                )
                            )
                        ).pipe(np.nanmedian)
                    ),
                    rtol=1e-10,
                    atol=1e-10,
                )

FileNotFoundError: There is no file named JR161-D20061118-T010645.raw

In [164]:
# Test Shape
array_shape = [6,6]
convolve_shape = [5,5]
radius_tuple = [
    convolve_shape[0]//2,
    convolve_shape[1]//2
]

# Create mock data
mock_data = np.random.rand(array_shape[0], array_shape[1])
da_mock_data = xr.DataArray(
    mock_data,
    dims=("row", "col"),
    name="random_numbers"
).chunk("auto")

In [165]:
# Symmetric matches the reflect logic in Dask Generic Filter
data_padded_with_reflection = np.pad(
    mock_data,
    radius_tuple,
    mode='symmetric'
)

In [166]:
# Use Echopype to calculate channel
output = dask_image.ndfilters.generic_filter(
    image=da_mock_data.data,
    function=np.nanmean,
    size=convolve_shape,
    mode="reflect",
).compute()

In [171]:
# Create mock data
padded_reflection_shape = data_padded_with_reflection.shape
output_expected = np.full(mock_data.shape, np.nan)

# Iterate through value
for i in range(radius_tuple[0], padded_reflection_shape[0] - radius_tuple[0]):
    for j in range(radius_tuple[1], padded_reflection_shape[1] - radius_tuple[1]):
        # Create window on data padded with reflection
        window = data_padded_with_reflection[
            (i-radius_tuple[0]):(i+radius_tuple[0] + 1),
            (j-radius_tuple[1]):(j+radius_tuple[1] + 1)
        ]

        # Calculate aggregate value of window
        agg_value = np.nanmean(window)

        # Place in output_expected
        output_expected[
            i-radius_tuple[0],
            j-radius_tuple[1]
        ] = agg_value


In [172]:
np.allclose(output, output_expected)

True